In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

from sklearn.metrics.pairwise import cosine_similarity


def get_user_id_from_pos(pos, df_full_ratings):
    return df_full_ratings.index[pos]

def get_movie_id_from_pos(pos, df_full_ratings):
    return df_full_ratings.columns[pos]

def get_title(movieId, df_movies):
    return df_movies.loc[movieId]["title"]
def get_seen_movie_ids(df_with_0_ratings, userId ):
    seen_movies = df_with_0_ratings.loc[userId]  
    seen_movies = seen_movies[  seen_movies != 0 ]
   
    return seen_movies.index.to_list()


def get_seen_movie_ids_with_ratings(df_with_0_ratings, userId):
    seen_movies = df_with_0_ratings.loc[userId]  
    seen_movies = seen_movies[  seen_movies != 0 ]
    
    #sorted_best_movies_pos = np.argsort( seen_movies.values )
    s =seen_movies.sort_values(ascending=False)
    return list(zip( s.index.to_list(), s.values   ))
def get_rmse(predictions , actual_vals):
        mse = 0.0
        
        x, y = actual_vals.nonzero()
        N = 0
        for i, j in zip(x, y):
            mse += pow(actual_vals[i][j] - predictions[i][j], 2)
            N += 1
       
        return np.sqrt(mse/N)
        



In [3]:
import pandas as pd
import numpy as np



ds_ratings_1m = "./movies_1m/ratings.dat"
ds_movies_1m = "./movies_1m/movies.dat"


df_mov = pd.read_csv( ds_movies_1m, sep="::", engine="python")
df_user_ratings = pd.read_csv( ds_ratings_1m, sep="::", engine="python" )




df_mov.columns =['movieId', 'title', 'genres']
df_mov.set_index("movieId", drop=True, inplace=True)
df_user_ratings.columns =  ['userId', 'movieId', 'rating', 'timestamp'] 


df_user_ratings = df_user_ratings.merge(df_mov, how="inner", on="movieId" )


df_num = df_user_ratings[["movieId", "userId"]].groupby(by="movieId").count()

df_num["num_ratings"] = df_num["userId"]
df_num.drop(columns="userId", axis=1,inplace=True)

df_user_ratings = df_user_ratings.merge(df_num, how="inner", on="movieId") 

min_ratings = 40
df_user_ratings = df_user_ratings[  df_user_ratings["num_ratings"] > min_ratings ]


ratings_mat = pd.pivot_table(df_user_ratings,index="userId", columns="movieId", values="rating" ).fillna(2.5)
ratings_mat_with_zeros = pd.pivot_table(df_user_ratings,index="userId", columns="movieId", values="rating" ).fillna(0)




R = ratings_mat.values
user_ratings_mean = np.mean(R, axis=1)
R_demeaned = R - user_ratings_mean.reshape(-1, 1)

        
            
        
        
        
from sklearn.decomposition import TruncatedSVD
from sklearn.random_projection import sparse_random_matrix

svd = TruncatedSVD(n_components=150, n_iter=7, random_state=42)
u_s = svd.fit_transform(R_demeaned)  

#print(svd.explained_variance_ratio_)  
print("Variance explained percent : ", svd.explained_variance_ratio_.sum())  
#print(svd.singular_values_) 

# return V transposed
vt = svd.components_

all_user_predicted_ratings = np.dot( u_s, vt  ) +  user_ratings_mean.reshape(-1, 1) 
preds_df = pd.DataFrame(all_user_predicted_ratings, index = ratings_mat.index , columns = ratings_mat.columns)



#rmse = get_rmse(all_user_predicted_ratings, ratings_mat.values)

rmse = get_rmse(all_user_predicted_ratings, ratings_mat_with_zeros.values)
print("rmse with sklearn")
print(rmse)

userId = 1

seen_movies_ratings =  df_user_ratings[ df_user_ratings["userId"] == userId ][  ["movieId", "rating"]]
seen_movie_ids = seen_movies_ratings["movieId"].values 
#print(seen_movie_ids)

preds_df.head()

predictions_for_user = preds_df.loc[userId].sort_values(ascending=False)
user_ratings = ratings_mat.loc[userId].sort_values(ascending=False)
movies_df = df_mov
i = 1
for movieId in predictions_for_user.index:
    if movieId not in seen_movie_ids:
        print( i, " : ", movies_df.loc[movieId]["title"], " ", predictions_for_user[movieId] )
        i += 1
        if i > 10:
            break
        





Variance explained percent :  0.48620726182946017
rmse with sklearn
0.888439397167944
1  :  Lion King, The (1994)   3.7975053323081602
2  :  Little Mermaid, The (1989)   3.447607303538021
3  :  Jungle Book, The (1967)   3.2391984697186285
4  :  Peter Pan (1953)   3.2239777661400155
5  :  Sleeping Beauty (1959)   3.211967704290644
6  :  Fantasia (1940)   3.1668895861074025
7  :  Lady and the Tramp (1955)   3.1342254339792084
8  :  Pinocchio (1940)   3.1218107244541833
9  :  Alice in Wonderland (1951)   3.116958387242011
10  :  101 Dalmatians (1961)   3.1098470411694965
